In [1]:
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(rc={"figure.figsize": (20, 10)})
plt.rc('figure', figsize=(20, 10))
import numpy as np
import utils;reload(utils)
from utils import *
from keras.layers import Merge, Permute
import pdb
from sklearn.model_selection import train_test_split
from scipy.stats.mstats import rankdata

/root/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.


In [2]:
data_dir = 'data_new/'

In [3]:
buy_cnn2d_trn = load_array(data_dir+'buy_cnn2d_trn')
buy_cnn2d_trn_label = load_array(data_dir+'buy_cnn2d_trn_label')
buy_cnn2d_cv = load_array(data_dir+'buy_cnn2d_cv')
buy_cnn2d_cv_label = load_array(data_dir+'buy_cnn2d_cv_label')

In [4]:
user_feature = utils.get_basic_user_feat(data_dir)

user_feature_dict = {}
for row in user_feature.values:
    user_feature_dict[row[0]] = row[1:]


In [5]:
users = load_array(data_dir+'users')

In [6]:
items = load_array(data_dir+'products')

In [7]:
buy_user_feat_trn = np.array([user_feature_dict[users[int(uidx)]] for uidx in buy_cnn2d_trn[:, 0, 0, 0]])
buy_user_feat_cv = np.array([user_feature_dict[users[int(uidx)]] for uidx in buy_cnn2d_cv[:, 0, 0, 0]])

In [8]:
cv_neg = buy_cnn2d_cv[buy_cnn2d_cv_label[:, 1] == 0]
# cv_pos_unused, cv_pos = train_test_split(buy_cnn2d_cv[buy_cnn2d_cv_label[:, 1] == 1], test_size=len(cv_neg)/40)
cv_pos = buy_cnn2d_cv[buy_cnn2d_cv_label[:, 1] == 1]
buy_cnn2d_cv_imitate = np.vstack((cv_pos, cv_neg))
buy_cnn2d_cv_imitate_label = np.vstack((np.ones((len(cv_pos), 2)), np.zeros((len(cv_neg), 2))))

In [9]:
buy_user_feat_cv_imitate = np.array([user_feature_dict[users[int(uidx)]] for uidx in buy_cnn2d_cv_imitate[:, 0, 0, 0]])

In [10]:
total_days = 10
days = 10
top_skus = 10
filter_day = 2
filter_sku = 2
days_offset = total_days-days

type_select = [1,2,3,5,6,7,8,9,10,11,12]
type_select_map = range(2, buy_cnn2d_trn.shape[-1])#[i-1+2 for i in type_select]
type_select_count = len(type_select_map)

In [11]:
trn = buy_cnn2d_trn[:, days_offset:, :top_skus, type_select_map]
cv = buy_cnn2d_cv_imitate[:, days_offset:, :top_skus, type_select_map]

In [12]:
buy_user_feat_cv_imitate

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ..., 
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [13]:
dtrain = xgb.DMatrix(np.hstack((trn.reshape(trn.shape[0], -1), buy_user_feat_trn)), missing = -999.0, label= buy_cnn2d_trn_label[:, 1])
dcv = xgb.DMatrix(np.hstack((cv.reshape(cv.shape[0], -1), buy_user_feat_cv_imitate)), missing= -999.0, label=buy_cnn2d_cv_imitate_label[:, 1])

In [40]:
watchlist = [ (dcv,'val'), (dtrain, 'train') ]

param = {'nthread': 8,
         'booster': 'gbtree',
         "objective": "binary:logistic",
         "eval_metric": "auc", "eta":  0.05,
         "max_depth": 1,
         "min_child_weight": 10,
         "gamma": 0.1,
         "lambda": 0.5,
         "alpha": 5,
         "scale_pos_weight": 0.5,
         "subsample": 0.8,
         'early_stopping_rounds':100}

num_round = 200
bst = xgb.train(param, dtrain, num_round, evals=watchlist)

[0]	val-auc:0.610181	train-auc:0.628912
[1]	val-auc:0.616674	train-auc:0.638452
[2]	val-auc:0.616674	train-auc:0.638452
[3]	val-auc:0.616095	train-auc:0.638517
[4]	val-auc:0.619657	train-auc:0.642533
[5]	val-auc:0.619657	train-auc:0.642533
[6]	val-auc:0.623	train-auc:0.643785
[7]	val-auc:0.623	train-auc:0.643785
[8]	val-auc:0.623	train-auc:0.643785
[9]	val-auc:0.623	train-auc:0.643785
[10]	val-auc:0.623	train-auc:0.643785
[11]	val-auc:0.640171	train-auc:0.6553
[12]	val-auc:0.641517	train-auc:0.655881
[13]	val-auc:0.646783	train-auc:0.662917
[14]	val-auc:0.646783	train-auc:0.662917
[15]	val-auc:0.639727	train-auc:0.664286
[16]	val-auc:0.639019	train-auc:0.665651
[17]	val-auc:0.6354	train-auc:0.665682
[18]	val-auc:0.637217	train-auc:0.665457
[19]	val-auc:0.637671	train-auc:0.665295
[20]	val-auc:0.636406	train-auc:0.665759
[21]	val-auc:0.635002	train-auc:0.666048
[22]	val-auc:0.635412	train-auc:0.666026
[23]	val-auc:0.634914	train-auc:0.666181
[24]	val-auc:0.635776	train-auc:0.665992
[25]

In [36]:
def f1(p, r):
    return 6 * p * r / (5 * r + p)
def f2(p, r):
    return 5 * p * r / (3 * p + 2 * r)

In [41]:
# buy_cnn2d_cv[:, :, :, 1:2], 
cv_pred = bst.predict(dcv)
cv_pred_df = pd.DataFrame(cv_pred)
# cv_pred_df = pd.DataFrame(np.argmax(cv_pred, 1))

In [44]:
cv_pred_df['user_id'] = [users[int(i)] for i in list(buy_cnn2d_cv_imitate[:, 0, 0, 0])]

cv_pred_df['buy'] = cv_pred_df.apply(lambda x: 1 if x[0] > 0.75 else 0, 1)

cv_pred_df['true'] = buy_cnn2d_cv_imitate_label[:, 1]
# cv_pred_df['true'] = [targets[i] for i in cnn_cv_label[:, 0]]

p = len(cv_pred_df[(cv_pred_df.buy == 1) & (cv_pred_df.true == 1)]) / float(len(cv_pred_df[cv_pred_df.buy==1]))
r = len(cv_pred_df[(cv_pred_df.buy == 1) & (cv_pred_df.true == 1)]) / float(len(cv_pred_df[cv_pred_df.true==1]))
# p = len(cv_pred_df[cv_pred_df.buy == cv_pred_df.true]) / float(len(cv_pred_df))
# r = len(cv_pred_df[cv_pred_df.buy == cv_pred_df.true]) / float(len(cv_pred_df))
p, r, f1(p, r)

(0.14035087719298245, 0.07017543859649122, 0.12030075187969924)

In [21]:
test = load_array(data_dir+'test_cnn2d')

In [22]:
test_user_feat = np.array([user_feature_dict[users[int(uidx)]] for uidx in test[:, 0, 0, 0]])

IndexError: index 53652 is out of bounds for axis 0 with size 50504

In [ ]:
test_matrix = test[:, days_offset:, :top_skus, type_select_map]

In [ ]:
dtest = xgb.DMatrix(np.hstack((test_matrix.reshape(test_matrix.shape[0], -1), test_user_feat)), missing = -999.0)

In [ ]:
buy_predicted = bst.predict(dtest)

In [ ]:
test_pred_df = pd.DataFrame(buy_predicted)
test_pred_df['user_id'] = [int(users[int(i)]) for i in list(test[:, 0, 0, 0])]
test_pred_df['is_buy'] = test_pred_df.apply(lambda x: 1 if x[0] > 0.7 else 0, 1)
test_pred_df['sku_id'] = 1
test_pred_df.is_buy.value_counts()

In [43]:
result_df = test_pred_df[(test_pred_df.is_buy==1)&(test_pred_df.sku_id!=0)][['user_id', 'sku_id']]
print len(result_df)
result_df.to_csv("xgb.csv", index=False)
from IPython.display import FileLink
FileLink('xgb.csv')

533


/home/zhli7/notebook/jd/xgb.csv

In [44]:
cnn = pd.read_csv('cnn2d.csv')

In [45]:
ensemble = pd.merge(result_df, cnn, on='user_id', how='inner')
len(ensemble)

71

In [46]:
ensemble['sku'] = ensemble.apply(lambda x: int(x['sku_id_x']) if math.isnan(x['sku_id_y']) else int(x['sku_id_y']), 1)

In [47]:
ensemble[['user_id', 'sku']].to_csv("ensemble.csv", index=False)
from IPython.display import FileLink
FileLink('ensemble.csv')

/home/zhli7/notebook/jd/ensemble.csv